In [28]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame 
import nltk

from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn import neighbors
from scipy.spatial.distance import cosine
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

import re
import string
from sklearn.metrics import mean_squared_error

In [29]:
df= pd.read_json("Automotive_5.json",lines = True)

In [30]:
df = df[['asin','overall','reviewerID']].copy()

In [31]:
from sklearn.model_selection import train_test_split
# Splitting train and test data at the start
df_train, df_test = train_test_split(df,test_size=0.2,stratify = df['reviewerID'])
df_train = df_train.sort_index(axis=0)

In [32]:
len(df_train)

16378

In [33]:
len(df_test)

4095

In [34]:
# df_original contains the copy of training sampy before pivoting 
df_original_train = df_train.copy()
df_train = df_train.pivot(index = "asin" , columns = "reviewerID" , values = "overall")
df_train = df_train.sort_index(axis=1)

In [35]:
df_train.shape

(1835, 2928)

In [36]:
df_test.pivot(index = "asin" , columns = "reviewerID" , values = "overall").shape

(1508, 2928)

In [37]:
# In this blockck we are normalizing each rows

X = df_train.values
Y = df_train.sum(axis = 1)
Z =df_train.count(axis =1).astype('float64')
Y = Y.div(Z,axis=0)
cosine
df_new_train = df_train.sub(Y,axis=0).fillna(0)

In [38]:
# fitting training data into our nearest neighbor model
nbrs_train = NearestNeighbors(n_neighbors= 5, algorithm='auto',metric ='cosine').fit(df_new_train.values)

In [39]:
# getting nmeighbors and their distances for each column
distances, indices = nbrs_train.kneighbors(df_new_train,return_distance=True)

In [40]:
distances

array([[0.00000000e+00, 9.11808290e-01, 9.45445527e-01, 9.53736362e-01,
        9.56259112e-01],
       [0.00000000e+00, 8.55662433e-01, 8.63069361e-01, 8.96194623e-01,
        9.22984595e-01],
       [0.00000000e+00, 8.33333333e-01, 8.80477139e-01, 8.87279628e-01,
        8.89903623e-01],
       ...,
       [2.22044605e-16, 6.27322004e-01, 8.79028324e-01, 9.19568916e-01,
        9.25273528e-01],
       [0.00000000e+00, 7.39103034e-01, 8.36015988e-01, 8.84045793e-01,
        9.30001262e-01],
       [0.00000000e+00, 8.53614989e-01, 8.66369379e-01, 8.71413875e-01,
        9.08996849e-01]])

In [41]:
indices

array([[   0,  494, 1127, 1054,  896],
       [   1, 1212, 1728,  177, 1229],
       [   2, 1681,  404,  209, 1374],
       ...,
       [1832, 1604, 1770, 1752, 1463],
       [1833,  169, 1173,  812,  755],
       [1834,  618,  366,  436, 1529]])

In [42]:
# df_train

In [43]:
# creating list for average ratings of all items
item_avg = df_original_train.groupby(['asin']).mean()
# item_avg

In [44]:
# creating list for average ratings for all users
user_avg = df_original_train.groupby(['reviewerID']).mean()
# user_avg

In [45]:
# calculating the mean average rating for all items and all dataset
mean_avg = df_original_train["overall"].mean()

In [46]:
# Now we have everything, we strat testing
df_test[4:5]

,asin,overall,reviewerID
1420,B0002U26QE,5,A3EGHUQPUV6E5U


In [47]:
# This item maps basically maps items with its index in training matrix so that based on that index we can get vectors from distances and indices

item_map = df_new_train.index.values
item_map.shape

(1835,)

temp

In [48]:
# basically we need to completely this loop
prediction_arr = []

checkbi = 0
for i in range(len(df_test)):
#     item = np.where(item_map == df_test[i:i+1]["asin"].iat[0])
#     item _ nbrs = indices[item[0]:item[0] + 1]
#     nbrs_distances = distances[item[0]:item[0] + 1]
    
    item_asin = df_test[i:i+1]['asin'].iat[0]
    item_userID = df_test[i:i+1]['reviewerID'].iat[0]
    actual_rating = df_test[i:i+1]['overall'].iat[0]
    
    predicted_rating = 0
    if item_asin in item_avg.index:
        bi = item_avg.loc[item_asin]['overall'] - mean_avg
        
    else:
        bi = 0
    if item_userID in user_avg.index:
        bx = user_avg.loc[item_userID]['overall'] - mean_avg
    else:
        bx = 0
            
    baseline_estimate = mean_avg + bi + bx
    predicted_rating = baseline_estimate
    
    numerator = 0 
    denomenator = 0
    
    if bi != 0:
        
        item  = np.where(item_map == df_test[i:i+1]["asin"].iat[0])[0][0]
        item_nbrs = indices[item:item+1]
        dist_nbrs = distances[item:item+1]
#         print(item_nbrs)
        cnt = 0;
        k = 0
        for j in item_nbrs:
            
            temp = df_original_train.loc[(df_original_train['asin'] == item_map[j[0]]) &  (df_original_train['reviewerID'] == item_userID)]
            if(temp.size != 0):
                checkbi+= 1
                dist = dist_nbrs[cnt:cnt+1]
#                 print(dist)
    #             print(temp['overall'].values[0])
                numerator += ((1 - dist[0][k])*(temp['overall'].values[0] - (item_avg.loc[temp['asin']]['overall'] - mean_avg) ))

                denomenator += (1 - dist[0][k])
    #             predicted_rating += (dist*temp['overall'].values[0])/dist
            cnt = cnt + 1
            k+= 1
        if denomenator:
            predicted_rating += numerator/denomenator 
    
    prediction_arr.append(predicted_rating)
#     print(predicted_rating)
#     print(actual_rating - predicted_rating)
    

In [49]:
actual_arr = df_test['overall'].values
mean_squared_error(actual_arr,prediction_arr)

0.9097443635548295

In [50]:
from math import sqrt

sqrt(mean_squared_error(actual_arr,prediction_arr))

0.9538052021009475

In [53]:
df_asin = df.groupby('asin').count()
df_asin = df_asin.index.values
len(df_asin)

1835

In [54]:
df_reviewerID = df.groupby('reviewerID').count()
df_reviewerID = df_reviewerID.index.values
len(df_reviewerID)

2928

In [55]:
df_dict = {}
for line in df_original_train.values:
    if line[0] in df_dict:
        df_dict[line[0]].append(line[2])
    else:
        df_dict[line[0]] = [line[2]]

In [56]:
predict_for_this = []
count = 0
for i in range(len(df_reviewerID)):
    for j in range(len(df_asin)):
#         print(df_asin[i])
        if df_asin[j] not  in  df_dict or not df_reviewerID[i] in df_dict[df_asin[j]]:
            predict_for_this.append((df_reviewerID[i],df_asin[j]))

In [ ]:
# predict_for_this = []
# for i in range(len(df_asin)):
#         if df_asin[i] not  in  df_dict or not "A100WO06OQR8BQ" in df_dict[df_asin[i]]:
#             predict_for_this.append(("A100WO06OQR8BQ",df_asin[i]))
# len(predict_for_this)

In [ ]:
# predict_for_this
final_prediction = {}

for i in range(len(predict_for_this)):
#__________________________________________________________________________________________________________
# for i in range(len(predict_for_this)): This loop needs to be run instead of above one. 5352407 records: takes time
#__________________________________________________________________________________________________________
    
    item_asin = predict_for_this[i][1]
#     print(item_asin)
    item_userID = predict_for_this[i][0]
#     print(item_asin,item_userID)
    
    predicted_rating = 0
    if item_asin in item_avg.index:
        bi = item_avg.loc[item_asin]['overall'] - mean_avg
    else:
        bi = 0
    if item_userID in user_avg.index:
        bx = user_avg.loc[item_userID]['overall'] - mean_avg
    else:
        bx = 0
            
    baseline_estimate = mean_avg + bi + bx
    predicted_rating = baseline_estimate
#     print(baseline_estimate)
    
    numerator = 0 
    denomenator = 0
    
    if bi != 0 :
        
        item  = np.where(item_map == item_asin)[0][0]
        item_nbrs = indices[item:item+1]
        dist_nbrs = distances[item:item+1]
        cnt = 0;
        k = 0
        
        for j in item_nbrs:
            temp = df_original_train.loc[(df_original_train['asin'] == item_map[j[0]]) &  (df_original_train['reviewerID'] == item_userID)]
            if(temp.size != 0):
                dist = dist_nbrs[cnt:cnt+1]
                numerator += ((1 - dist[0][k])*(temp['overall'].values[0] - (item_avg.loc[temp['asin']]['overall'] - mean_avg) ))

                denomenator += (1 - dist[0][k])
            cnt = cnt + 1
            k+= 1
        if denomenator:
            predicted_rating += numerator/denomenator 

    if item_userID in final_prediction:
        final_prediction[item_userID].append((item_asin,predicted_rating))
    else:
        final_prediction[(item_userID)] = [(item_asin,predicted_rating)]

#___________________________________________________________
#        Sort final_prediction on basis of rating
#___________________________________________________________



In [ ]:
import json
with open('prediction.json', 'w') as fp:
    json.dump(final_prediction, fp)

In [ ]:
# Getting top 10 recommendation
for x in final_prediction:
    final_prediction[x] = sorted(final_prediction[x],key = lambda x:x[1],reverse= True)[:10]

In [ ]:
precision_count = 0
recall_count = 0
avg_precision = 0
avg_recall = 0
for i in user_list:
    precision_count = 0
    recall_count = 0
    for j in range(10):
        if final_prediction[i][j] in df_test['asin'].values:
            precision_count += 1
        if final_prediction[i][j] in (df_test.loc[df_test['reviewerID']== i]):
            recall_count += 1
                                                    
    user_precision = precision_count/10
    avg_precision+=user_precision
    user_recall = recall_count/(len(df_test.loc[df_test['reviewerID'] == i]))
    avg_recall+= user_recall
pr = avg_precision/(len(user_list))
re = avg_recall/(len(user_list))
print("Precision: ")
print(pr)
print("Recall: ")
print(re)
print("F-measure: ")
print(2*(pr)*(re)/(pr+re))

